In [1]:
import sys
from time import sleep
import argparse
import requests  # for calling api
import logging
import pandas as pd
import json
import os.path
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 180)
pd.set_option('display.max_columns', 20)

In [2]:
api_stations = f"https://geo.irceline.be/sos/api/v1/stations/?expanded=false"
api_stations_exp = "https://geo.irceline.be/sos/api/v1/stations/?expanded=true"
api_stations_gent = f"https://geo.irceline.be/sos/api/v1/stations/1207?expanded=true"

In [3]:
def data_from_api(url, norm):
    response_status = requests.get(url)
    response = response_status.json()
    if isinstance(response, dict): print(response)
    else: print(response[0])
    if norm: df = pd.json_normalize(response)
    else: df = pd.DataFrame(response)
    return df

In [4]:
stations = data_from_api(api_stations, norm=True)

{'properties': {'id': 1030, 'label': '40AL01 - Linkeroever'}, 'geometry': {'coordinates': [4.385223684454717, 51.23619419990248, 'NaN'], 'type': 'Point'}, 'type': 'Feature'}


In [36]:
station_info = pd.concat([pd.json_normalize(stations.properties), pd.json_normalize(stations.geometry)], axis=1).drop("type", axis=1)
station_info

,id,label,coordinates
0,1030,40AL01 - Linkeroever,"[4.385223684454717, 51.23619419990248, NaN]"
1,1031,40AL02 - Beveren,"[4.234832753144059, 51.30452079034428, NaN]"
2,1032,40AL03 - Beveren,"[4.201460395126572, 51.25396488066997, NaN]"
3,1033,40AL04 - Beveren,"[4.293329476940385, 51.2906675267468, NaN]"
4,1034,40AL05 - Beveren,"[4.278889821667828, 51.2631177168737, NaN]"
...,...,...,...
118,1241,47E716 - Mariakerke,"[3.6828130172031206, 51.06698790152168, NaN]"
119,1716,47E814 - Ham,"[5.129718074812025, 51.08218722351412, NaN]"
120,1752,48R515 - Aeroport 1,"[4.425464746587246, 50.45160018054414, NaN]"
121,1753,48R516 - Aeroport 2,"[4.484957959246502, 50.46913160387411, NaN]"


In [5]:
data_from_api(api_stations_exp, norm=False)

{'properties': {'timeseries': {'6152': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}, 'offering': {'id': '6152', 'label': '6152 - DAILY CORRECTION TEOM - procedure'}, 'feature': {'id': '1030', 'label': '40AL01 - Linkeroever'}, 'procedure': {'id': '6152', 'label': '6152 - DAILY CORRECTION TEOM - procedure'}, 'phenomenon': {'id': '5', 'label': 'Particulate Matter < 10 µm'}, 'category': {'id': '5', 'label': 'Particulate Matter < 10 µm'}}, '6151': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}, 'offering': {'id': '6151', 'label': '6151 - Unknown device - procedure'}, 'feature': {'id': '1030', 'label': '40AL01 - Linkeroever'}, 'procedure': {'id': '6151', 'label': '6151 - Unknown device - procedure'}, 'phenomenon': {'id': '391', 'label': 'Black Carbon'}, 'category': {'id': '391', 'label': 'Black Carbon'}}, '10897': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}, 'offering': {'id': '10897', 'label': '10

,properties,geometry,type
0,"{'timeseries': {'6152': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2....","{'coordinates': [4.385223684454717, 51.23619419990248, 'NaN'], 'type': 'Point'}",Feature
1,"{'timeseries': {'6153': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2....","{'coordinates': [4.234832753144059, 51.30452079034428, 'NaN'], 'type': 'Point'}",Feature
2,"{'timeseries': {'10767': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2...","{'coordinates': [4.201460395126572, 51.25396488066997, 'NaN'], 'type': 'Point'}",Feature
3,"{'timeseries': {'10844': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2...","{'coordinates': [4.293329476940385, 51.2906675267468, 'NaN'], 'type': 'Point'}",Feature
4,"{'timeseries': {'10845': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2...","{'coordinates': [4.278889821667828, 51.2631177168737, 'NaN'], 'type': 'Point'}",Feature
...,...,...,...
118,"{'timeseries': {'7206': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2....","{'coordinates': [3.6828130172031206, 51.06698790152168, 'NaN'], 'type': 'Point'}",Feature
119,"{'timeseries': {'10707': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2...","{'coordinates': [5.129718074812025, 51.08218722351412, 'NaN'], 'type': 'Point'}",Feature
120,"{'timeseries': {'100041': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS ...","{'coordinates': [4.425464746587246, 50.45160018054414, 'NaN'], 'type': 'Point'}",Feature
121,"{'timeseries': {'100050': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS ...","{'coordinates': [4.484957959246502, 50.46913160387411, 'NaN'], 'type': 'Point'}",Feature


In [6]:
# get timeseries ids for parameters of interest from station gent
station_gent = data_from_api(api_stations_gent, norm=False)
timeseries_gent = pd.DataFrame(station_gent.loc["timeseries", "properties"]).transpose().reset_index(names="timeseries_id")

# get timeseries ids for per station






{'properties': {'timeseries': {'7087': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}, 'offering': {'id': '7087', 'label': '7087 - FIDAS 200 - procedure'}, 'feature': {'id': '1207', 'label': '44R701 - Gent'}, 'procedure': {'id': '7087', 'label': '7087 - FIDAS 200 - procedure'}, 'phenomenon': {'id': '6001', 'label': 'Particulate Matter < 2.5 µm'}, 'category': {'id': '6001', 'label': 'Particulate Matter < 2.5 µm'}}, '99906': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}, 'offering': {'id': '99906', 'label': '99906 - Unknown device - procedure'}, 'feature': {'id': '1207', 'label': '44R701 - Gent'}, 'procedure': {'id': '99906', 'label': '99906 - Unknown device - procedure'}, 'phenomenon': {'id': '61102', 'label': 'Wind Direction'}, 'category': {'id': '61102', 'label': 'Wind Direction'}}, '7078': {'service': {'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}, 'offering': {'id': '7078', 'label': '7078 - Env. O341M tot

In [37]:
timeseries_gent

,timeseries_id,service,offering,feature,procedure,phenomenon,category
0,7087,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7087', 'label': '7087 - FIDAS 200 - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7087', 'label': '7087 - FIDAS 200 - procedure'}","{'id': '6001', 'label': 'Particulate Matter < 2.5 µm'}","{'id': '6001', 'label': 'Particulate Matter < 2.5 µm'}"
1,99906,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '99906', 'label': '99906 - Unknown device - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '99906', 'label': '99906 - Unknown device - procedure'}","{'id': '61102', 'label': 'Wind Direction'}","{'id': '61102', 'label': 'Wind Direction'}"
2,7078,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7078', 'label': '7078 - Env. O341M tot 20041114, nu API400E - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7078', 'label': '7078 - Env. O341M tot 20041114, nu API400E - procedure'}","{'id': '7', 'label': 'Ozone'}","{'id': '7', 'label': 'Ozone'}"
3,7080,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7080', 'label': '7080 - Airmotec 1000 - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7080', 'label': '7080 - Airmotec 1000 - procedure'}","{'id': '20', 'label': 'Benzene'}","{'id': '20', 'label': 'Benzene'}"
4,7071,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7071', 'label': '7071 - API300 - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7071', 'label': '7071 - API300 - procedure'}","{'id': '10', 'label': 'Carbon Monoxide'}","{'id': '10', 'label': 'Carbon Monoxide'}"
5,7073,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7073', 'label': '7073 - THIS 42C - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7073', 'label': '7073 - THIS 42C - procedure'}","{'id': '38', 'label': 'Nitrogen monoxide'}","{'id': '38', 'label': 'Nitrogen monoxide'}"
6,7072,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7072', 'label': '7072 - ENVIRONNEMENT AF21M - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7072', 'label': '7072 - ENVIRONNEMENT AF21M - procedure'}","{'id': '1', 'label': 'Sulphur dioxide'}","{'id': '1', 'label': 'Sulphur dioxide'}"
7,7086,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7086', 'label': '7086 - FIDAS 200 - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7086', 'label': '7086 - FIDAS 200 - procedure'}","{'id': '5', 'label': 'Particulate Matter < 10 µm'}","{'id': '5', 'label': 'Particulate Matter < 10 µm'}"
8,7074,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '7074', 'label': '7074 - THIS 42C - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '7074', 'label': '7074 - THIS 42C - procedure'}","{'id': '8', 'label': 'Nitrogen dioxide'}","{'id': '8', 'label': 'Nitrogen dioxide'}"
9,10602,"{'id': '1', 'label': 'IRCEL - CELINE: timeseries-api (SOS 2.0)'}","{'id': '10602', 'label': '10602 - - procedure'}","{'id': '1207', 'label': '44R701 - Gent'}","{'id': '10602', 'label': '10602 - - procedure'}","{'id': '391', 'label': 'Black Carbon'}","{'id': '391', 'label': 'Black Carbon'}"


In [11]:
# get timeseries metadata for those timeseries available in station of interest
def get_timeseries_meta(ts_id):
    response_status = requests.get(f"https://geo.irceline.be/sos/api/v1/timeseries/{ts_id}")
    response = response_status.json()
    try:
        os.makedirs("../local_data/metadata")
    except Exception:
        pass
    fn = f"../local_data/metadata/{ts_id}.json"
    with open(fn, 'w') as fp:
        json.dump(response, fp)
    # timeseries_meta = pd.json_normalize(response)  # 1 rij voor 1 timeseries, maar sommige timeseries hebben 26 kolommen sommige 27

In [12]:
for ts_id in timeseries_gent.timeseries_id:
    get_timeseries_meta(ts_id)

In [13]:
# "https://geo.irceline.be/sos/api/v1/timeseries/7087"
# https://geo.irceline.be/sos/api/v1/timeseries/7087/getData?timespan=PT24H/2023-11-27
# get timeseries datapoints

def get_timeseries_datapoints(ts_id, date="2023-08-01"):
    date = date
    datefn = date.replace("-", "")
    timespan = f"?timespan=PT24H/{date}"
    #timespan = ""
    response_status = requests.get(f"https://geo.irceline.be/sos/api/v1/timeseries/{ts_id}/getData{timespan}")
    response = response_status.json()
    df = pd.DataFrame(response)
    df = pd.DataFrame(df['values'].values.tolist())
    try:
        os.makedirs(f"../local_data/timeseries_data/{datefn}/Gent")
    except Exception:
        pass
    df.to_csv(f"../local_data/timeseries_data/{datefn}/Gent/{ts_id}_data.txt", sep="\t", index=False)

In [14]:
dates = ["2023-08-01", "2023-08-02", "2023-08-03", "2023-08-04"]

for date in dates:
    for ts_id in timeseries_gent.timeseries_id:
        get_timeseries_datapoints(ts_id, date)

In [15]:
stations = data_from_api(api_stations, norm=False)

{'properties': {'id': 1030, 'label': '40AL01 - Linkeroever'}, 'geometry': {'coordinates': [4.385223684454717, 51.23619419990248, 'NaN'], 'type': 'Point'}, 'type': 'Feature'}


In [16]:
stations

,properties,geometry,type
0,"{'id': 1030, 'label': '40AL01 - Linkeroever'}","{'coordinates': [4.385223684454717, 51.23619419990248, 'NaN'], 'type': 'Point'}",Feature
1,"{'id': 1031, 'label': '40AL02 - Beveren'}","{'coordinates': [4.234832753144059, 51.30452079034428, 'NaN'], 'type': 'Point'}",Feature
2,"{'id': 1032, 'label': '40AL03 - Beveren'}","{'coordinates': [4.201460395126572, 51.25396488066997, 'NaN'], 'type': 'Point'}",Feature
3,"{'id': 1033, 'label': '40AL04 - Beveren'}","{'coordinates': [4.293329476940385, 51.2906675267468, 'NaN'], 'type': 'Point'}",Feature
4,"{'id': 1034, 'label': '40AL05 - Beveren'}","{'coordinates': [4.278889821667828, 51.2631177168737, 'NaN'], 'type': 'Point'}",Feature
...,...,...,...
118,"{'id': 1241, 'label': '47E716 - Mariakerke'}","{'coordinates': [3.6828130172031206, 51.06698790152168, 'NaN'], 'type': 'Point'}",Feature
119,"{'id': 1716, 'label': '47E814 - Ham'}","{'coordinates': [5.129718074812025, 51.08218722351412, 'NaN'], 'type': 'Point'}",Feature
120,"{'id': 1752, 'label': '48R515 - Aeroport 1'}","{'coordinates': [4.425464746587246, 50.45160018054414, 'NaN'], 'type': 'Point'}",Feature
121,"{'id': 1753, 'label': '48R516 - Aeroport 2'}","{'coordinates': [4.484957959246502, 50.46913160387411, 'NaN'], 'type': 'Point'}",Feature


,id,label,coordinates
0,1030,40AL01 - Linkeroever,"[4.385223684454717, 51.23619419990248, NaN]"
1,1031,40AL02 - Beveren,"[4.234832753144059, 51.30452079034428, NaN]"
2,1032,40AL03 - Beveren,"[4.201460395126572, 51.25396488066997, NaN]"
3,1033,40AL04 - Beveren,"[4.293329476940385, 51.2906675267468, NaN]"
4,1034,40AL05 - Beveren,"[4.278889821667828, 51.2631177168737, NaN]"
...,...,...,...
118,1241,47E716 - Mariakerke,"[3.6828130172031206, 51.06698790152168, NaN]"
119,1716,47E814 - Ham,"[5.129718074812025, 51.08218722351412, NaN]"
120,1752,48R515 - Aeroport 1,"[4.425464746587246, 50.45160018054414, NaN]"
121,1753,48R516 - Aeroport 2,"[4.484957959246502, 50.46913160387411, NaN]"
